In [1]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
import  pandas as pd

import faiss
import pandas as pd
import file_utils
import document_db
from project_path import PROJECT_PATH
from annoy import AnnoyIndex


D:\projects\searchEngine\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


project_path.py D:\projects\searchEngine


In [2]:
index = AnnoyIndex(dimension, 'dot')  # Using 'angular' distance


NameError: name 'dimension' is not defined

In [11]:
file = r'C:\Users\lenovo\Downloads\use_cases_file_12.csv'

In [12]:
df = pd.read_csv(file)

In [18]:
def text_df_from_df(df, text_columns,id_column):
    cols2_df = []
    for str_col in text_columns:
        df["col_name"] = str_col
        str_col2_indexs = df.groupby([str_col,'col_name'])[id_column].apply(list).reset_index()
        str_col2_indexs = str_col2_indexs.rename(columns={str_col: 'text', 'index': 'document_ids'})
        cols2_df.append(str_col2_indexs)
    text_indexes_df = pd.concat(cols2_df)
    return text_indexes_df


In [25]:
# df_reindex = df.reset_index()



In [26]:
text_indexes_df

,text,col_name,Serial Number
0,5-day tech transfer and marketing platform dep...,Use Case Name,[26]
1,Acquisition ITA with property data append,Use Case Name,[52]
2,Application pipeline analysis to forecast fund...,Use Case Name,[37]
3,Auto updating CRM contacts daily for personali...,Use Case Name,[18]
4,Bid portfolio pricing based on runoff and reca...,Use Case Name,[40]
...,...,...,...
50,Target Now,CTA Text,[5]
51,Track Applications,CTA Text,[16]
52,Track Attribution,CTA Text,[17]
53,Track Now,CTA Text,[15]


In [21]:
text_df_from_df(df,text_columns=text_columns)

,text,document_ids
0,5-day tech transfer and marketing platform dep...,[25]
1,Acquisition ITA with property data append,[51]
2,Application pipeline analysis to forecast fund...,[36]
3,Auto updating CRM contacts daily for personali...,[17]
4,Bid portfolio pricing based on runoff and reca...,[39]
...,...,...
50,Target Now,[4]
51,Track Applications,[15]
52,Track Attribution,[16]
53,Track Now,[14]


In [20]:
text_columns = [ 'Use Case Name', 'Use Case Category', 'Description',
       'Solution text', 'Result Text', 'CTA Text']
id_column = "Serial Number"


In [16]:
df.columns

Index(['Serial Number', 'Use Case Name', 'Use Case Category', 'Description',
       'Solution text', 'Result Text', 'CTA Text', 'Post_DI'],
      dtype='object')

In [27]:
df_reindex = df.reset_index()
cols2_df = []
for str_col in ['Title','Description']:
    str_col2_indexs = df_reindex.groupby(str_col)['index'].apply(list).reset_index()
    str_col2_indexs = str_col2_indexs.rename(columns={str_col:'text','index':'document_ids'})
    cols2_df.append(str_col2_indexs)
text_indexes_df = pd.concat(cols2_df)


In [34]:
text_indexes_df

,text,document_ids
0,5-day tech transfer and marketing platform dep...,[25]
1,Acquisition ITA with property data append,[51]
2,Application pipeline analysis to forecast fund...,[36]
3,Auto updating CRM contacts daily for personali...,[17]
4,Bid portfolio pricing based on runoff and reca...,[39]
...,...,...
32,Strategic Broker Targeting for Market Expansion,[4]
33,Strategic Pipeline Management to Forecast and ...,[36]
34,Streamlined Campaign Management Across Multipl...,[28]
35,Targeted Communication Based on Credit Activit...,[26]


In [15]:
import os

def delete_file(file_path):
    """
    Deletes a file from the specified file path.

    Args:
        file_path (str): The path to the file to be deleted.

    Returns:
        bool: True if the file was successfully deleted, False if the file does not exist.
    """
    if os.path.isfile(file_path):
        os.remove(file_path)
        print(f"File {file_path} deleted successfully.")
        return True
    else:
        print(f"File {file_path} does not exist.")
        return False


# Example usage
# file_path = 'path/to/your/file.txt'
# delete_file(file_path)
dimension

768

In [16]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

dimension = model.config.hidden_size 
index = AnnoyIndex(dimension, 'dot')  # Using 'angular' distance

In [17]:
path = r"D:\projects\searchEngine\data\vector_dbs\Ketul.nns"

index.load(path)

True

In [103]:
help(index.save)

Help on built-in function save:

save(...) method of annoy.Annoy instance
    Saves the index to disk.



In [87]:
vectors.shape

(102, 384)

In [85]:
vectors = texts_to_embeddings(text_indexes_df['text'].to_list())


In [89]:

from annoy import AnnoyIndex
dimension = vectors.shape[1]
# ["angular", "euclidean", "manhattan", "hamming", "dot]
index = AnnoyIndex(dimension, 'dot')  # Using 'angular' distance
# index = AnnoyIndex(dimension, 'euclidean')  # Using 'angular' distance
for i, vec in enumerate(vectors):
    index.add_item(i, vec)
index.build(30)
# index.save(index_path)

text = "Call Centers"
vector = text_to_vector(text).reshape((-1,1))

indices = index.get_nns_by_vector(vector,n=10, include_distances=True)  # Find 10 nearest neighbors
indexes,distances = indices
sel_df = text_indexes_df.iloc[indexes]
sel_df['distance'] = distances
sel_df.to_dict(orient='records')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_13584\3109577587.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  indices = index.get_nns_by_vector(vector,n=10, include_distances=True)  # Find 10 nearest neighbors
C:\Users\lenovo\AppData\Local\Temp\ipykernel_13584\3109577587.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_df['distance'] = distances


[{'text': 'Real-time credit trigger integration for Call Centers',
  'document_ids': [24],
  'distance': 26.671525955200195},
 {'text': 'Call Center app to provide loan officers with borrower offer details',
  'document_ids': [20],
  'distance': 21.824918746948242},
 {'text': 'Call log transcripts modeling from servicing and originations',
  'document_ids': [53],
  'distance': 17.003925323486328},
 {'text': 'Programming IVR with loan pricing for call routing',
  'document_ids': [18],
  'distance': 15.611470222473145},
 {'text': 'Optimizing Call Routing with Real-Time Loan Pricing in IVR Systems',
  'document_ids': [18],
  'distance': 13.70603084564209},
 {'text': 'Personalized direct mail messaging',
  'document_ids': [7],
  'distance': 13.39556884765625},
 {'text': 'Direct Mail campaign for mortgage health checkup',
  'document_ids': [21],
  'distance': 12.956390380859375},
 {'text': 'Borrower benefit segmentation',
  'document_ids': [6],
  'distance': 11.637833595275879},
 {'text': '

,text,document_ids
11,Empowering Borrowers with Comprehensive Curren...,[21]
16,Cross-selling mortgage to other customers,[43]
38,Maximizing borrower benefit through no PI incr...,[9]
13,Enhancing Borrower Communication with Personal...,[22]
40,Multi-product borrower loan scenario bulk pricing,[5]
35,Targeted Communication Based on Credit Activit...,[26]
16,Enhancing Personalization through Borrower Ben...,[6]
19,Enabling servicing agents to use loan data for...,[19]
43,Paid-off loans competitor analysis for share o...,[40]
54,Target list to prospect brokers for wholesale ...,[4]


384

AttributeError: 'Embedding' object has no attribute 'shape'

In [83]:
import numpy as np
from transformers import AutoTokenizer, AutoModel

import faiss
import pandas as pd
import file_utils
import document_db
from project_path import PROJECT_PATH

# "distilbert-base-uncased"

model = "sentence-transformers/all-MiniLM-L6-v2"
# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModel.from_pretrained(model)
dimension = 768  # Dimension of embeddings from DistilBERT
index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity search

# Example usage
db_manager = document_db.DocumentManager('my_documents.db')


def texts_to_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()


def text_to_vector(text):
    """Converts input text to a vector using DistilBERT."""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    # Use mean pooling to convert the output to a single vector per input text
    embeddings = outputs.last_hidden_state.mean(1).detach().numpy()
    return embeddings



D:\projects\searchEngine\venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lenovo\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
text = 'Broker sentiment'

In [8]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import faiss
import document_db

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")
dimension = 768  # Dimension of embeddings from DistilBERT
index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity search

db_manager = document_db.DocumentManager('my_documents.db')  # Database manager

def text_to_vector(text):
    """Converts input text to a vector using DistilBERT."""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    outputs = model(**inputs)
    # Use mean pooling to convert the output to a single vector per input text
    return outputs.last_hidden_state.mean(1).detach().numpy()

def load_and_index_data(filepath):
    """Load data from a CSV, convert texts to vectors, and index them in FAISS."""
    df = pd.read_csv(filepath)
    print(df.columns)
    for text in df['Title']:
        vector = text_to_vector(text)
        index.add(vector)  # Add the vector to FAISS index

def find_similar_texts(query_text, k=5):
    """Find k similar texts in the index given a query text."""
    query_vector = text_to_vector(query_text)
    D, I = index.search(query_vector, k)  # Perform the search
    return D, I


In [1]:
# Assume you have a CSV file 'data.csv' with a column 'text' containing the texts.
load_and_index_data(r'C:\Users\lenovo\Downloads\use_cases.csv')


# Query for similar texts
distances, indices = find_similar_texts("example query text")
print("Distances:", distances)
print("Indices:", indices)



NameError: name 'load_and_index_data' is not defined

In [4]:
vector = text_to_vector(text)
print("text to vector done")
# Search in FAISS index; k is the number of nearest neighbors you want


text to vector done


In [7]:
index.d

768

In [6]:
help(index.search)

Help on method replacement_search in module faiss.class_wrappers:

replacement_search(x, k, *, params=None, D=None, I=None) method of faiss.swigfaiss_avx2.IndexFlatL2 instance
    Find the k nearest neighbors of the set of vectors x in the index.
    
    Parameters
    ----------
    x : array_like
        Query vectors, shape (n, d) where d is appropriate for the index.
        `dtype` must be float32.
    k : int
        Number of nearest neighbors.
    params : SearchParameters
        Search parameters of the current search (overrides the class-level params)
    D : array_like, optional
        Distance array to store the result.
    I : array_like, optional
        Labels array to store the results.
    
    Returns
    -------
    D : array_like
        Distances of the nearest neighbors, shape (n, k). When not enough results are found
        the label is set to +Inf or -Inf.
    I : array_like
        Labels of the nearest neighbors, shape (n, k).
        When not enough resul

In [1]:
vector

NameError: name 'vector' is not defined

In [ ]:
index.search(vector, k=2)

In [ ]:
D = index.search(np.array([vector]), k=10)


In [1]:
D

NameError: name 'D' is not defined